In [1]:
# Display plots in the notebook
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [18]:
# Common imports
import numpy as np
import pylab as pl
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [3]:
# Make plots prettyful
pl.style.use('fivethirtyeight')

In [82]:
data = pd.read_csv('hw_2_stops.csv', dtype={'precinct': str, 'time.period': str})

In [83]:
data.columns

Index(['suspected.crime', 'found.weapon', 'found.gun', 'arrested', 'frisked',
       'searched', 'summons.issued', 'found.contraband', 'force.hands',
       'force.wall', 'force.ground', 'force.drawn', 'force.pointed',
       'force.baton', 'force.handcuffs', 'force.pepper', 'force.other',
       'suspect.race', 'suspect.age', 'suspect.build', 'suspect.sex',
       'suspect.height', 'suspect.weight', 'stopped.bc.desc',
       'stopped.bc.violent', 'stopped.bc.other', 'stopped.bc.object',
       'stopped.bc.casing', 'stopped.bc.lookout', 'stopped.bc.drugs',
       'stopped.bc.clothing', 'stopped.bc.furtive', 'stopped.bc.bulge',
       'precinct', 'inside.outside', 'location.housing', 'observation.period',
       'officer.uniform', 'additional.report', 'additional.investigation',
       'additional.proximity', 'additional.evasive', 'additional.associating',
       'additional.direction', 'additional.highcrime', 'additional.time',
       'additional.sights', 'additional.other', 'radio.run

In [84]:
data = data.dropna()

In [85]:
target = data['found.weapon']

In [86]:
X = data[[c for c in data.columns if 'found' not in c and 'force' not in c and c not in ['arrested', 'frisked', 'searched', 'date', 'summons.issued']]]

In [87]:
X = pd.get_dummies(X)

In [88]:
X_train, X_test, target_train, target_test = train_test_split(
    X, target, test_size=0.25, random_state=1
)

In [175]:
clf = RandomForestClassifier(n_jobs=-1, n_estimators=1000)
clf = clf.fit(X_train, target_train)

In [176]:
pred = clf.predict(X_train)
print("Classification accuracy for train set =", 1.0*sum(target_train==pred)/len(pred))

pred = clf.predict(X_test)
print("Classification accuracy for test set =", 1.0*sum(target_test==pred)/len(pred))

Classification accuracy for train set = 0.999995339885
Classification accuracy for test set = 0.988920616391


In [198]:
print("Test true precision", sum(target_test&pred)/sum(pred))
print("Test true recall", sum(target_test&pred)/sum(target_test))

print("Test false precision", sum((~target_test)&(~pred))/sum(~pred))
print("Test false recall", sum((~target_test)&(~pred))/sum(~target_test))

Test true precision 0.748
Test true recall 0.109420713868
Test false precision 0.989342371077
Test false recall 0.999554296266


In [199]:
# Most important features
X.columns[np.argsort(clf.feature_importances_)[::-1]]

Index(['lat', 'lon', 'suspect.age', 'suspect.weight', 'suspect.height',
       'suspected.crime_cpw', 'stopped.bc.object', 'observation.period',
       'stopped.bc.bulge', 'additional.time',
       ...
       'suspected.crime_unlawful use of credit card, debit card',
       'suspected.crime_public display of offensive sexual material',
       'suspected.crime_tampering with a public record',
       'suspected.crime_substitution of children',
       'suspected.crime_unlawful duplication of computer material',
       'suspected.crime_unauthorized use of computer',
       'suspected.crime_unlawful wearing a body vest',
       'suspected.crime_rent gouging',
       'suspected.crime_aggravated sexual abuse', 'suspected.crime_adultery'],
      dtype='object', length=254)

In [200]:
prob = clf.predict_proba(X_test)

In [170]:
prob.sum(axis=0)

array([ 282703.84166662,    3413.15833333])

In [167]:
X_test[c] == 2.0

826485     False
225511     False
1154777    False
330794     False
75099      False
1023179    False
549484     False
300111     False
1183957    False
492449     False
521475     False
929278     False
45626      False
938501     False
1141558    False
181606     False
1002717    False
566323     False
1160013    False
263324     False
1163407    False
247310     False
207279     False
21122      False
192196     False
252511     False
775539     False
886475     False
1083048    False
932959     False
           ...  
474643     False
685886     False
500004     False
413645     False
134643     False
562378     False
614307     False
1051342    False
802275     False
10332      False
72926      False
1069560    False
1054995    False
875790     False
756789     False
477128     False
1017756    False
1073731    False
509917     False
962063     False
652784     False
511054     False
553752     False
449291     False
310436     False
829747     False
487533     False
645074     Fal

In [166]:
np.array(prob)[X_test[c] == 2.0]

c:\Users\docmario\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  if __name__ == '__main__':


array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [157]:
prob[X_test[c] == 2.0]

c:\Users\docmario\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  if __name__ == '__main__':


array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [135]:
prob[X_test[c]].shape

c:\Users\docmario\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: non integer (and non boolean) array-likes will not be accepted as indices in the future
  if __name__ == '__main__':


(286117, 2)

In [174]:
cols = [c for c in X.columns if 'suspect.race' in c]
print(cols)
for c in cols:
    print(prob[np.where(X_test[c] == 1)].mean(axis=0))

['suspect.race_asian', 'suspect.race_black', 'suspect.race_hispanic', 'suspect.race_native.american', 'suspect.race_other', 'suspect.race_white']
[ 0.98935967  0.01064033]
[ 0.98920212  0.01079788]
[ 0.98805535  0.01194465]
[ 0.98899108  0.01100892]
[ 0.98776844  0.01223156]
[ 0.98104265  0.01895735]


In [112]:
X_test[X_test[cols[1]] == 1].head(10)

,suspect.age,suspect.height,suspect.weight,stopped.bc.desc,stopped.bc.violent,stopped.bc.other,stopped.bc.object,stopped.bc.casing,stopped.bc.lookout,stopped.bc.drugs,...,month_May,month_November,month_October,month_September,time.period_1,time.period_2,time.period_3,time.period_4,time.period_5,time.period_6
826485,17,6.000000,150.0,False,True,True,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
225511,23,5.250000,140.0,False,False,False,False,True,True,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
75099,29,6.083333,190.0,False,True,False,False,False,False,False,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
549484,14,5.833333,160.0,False,False,False,False,False,False,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
300111,22,6.166667,180.0,True,False,False,False,False,False,False,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1183957,18,5.583333,140.0,False,False,True,False,False,False,False,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
521475,23,5.500000,180.0,False,True,False,False,True,False,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
181606,29,5.666667,160.0,False,False,True,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
566323,22,5.333333,110.0,False,True,False,True,False,False,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
263324,25,6.166667,200.0,False,False,True,False,False,False,False,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [113]:
target_test[X_test[cols[1]] == 1].head(10)

826485     False
225511      True
75099      False
549484     False
300111     False
1183957    False
521475     False
181606     False
566323     False
263324     False
Name: found.weapon, dtype: bool

In [115]:
prob[X_test[cols[1]] == 1][:10]

c:\Users\docmario\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  if __name__ == '__main__':


array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [116]:
pred[X_test[cols[1]] == 1][:10]

c:\Users\docmario\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  if __name__ == '__main__':


array([False, False, False, False, False, False, False, False, False, False], dtype=bool)

In [118]:
pred.sum()

588

In [121]:
target_train.sum()

9976

In [38]:
#X.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
X.dropna(axis=0)

,summons.issued,suspect.age,suspect.height,suspect.weight,stopped.bc.desc,stopped.bc.violent,stopped.bc.other,stopped.bc.object,stopped.bc.casing,stopped.bc.lookout,...,month_December,month_February,month_January,month_July,month_June,month_March,month_May,month_November,month_October,month_September
0,False,21,5.666667,182.0,False,False,False,False,True,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,False,15,5.500000,140.0,False,False,False,False,True,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,False,17,5.583333,130.0,False,False,True,False,False,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,False,17,5.583333,130.0,False,False,True,False,False,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,False,20,6.250000,185.0,False,False,True,False,False,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,False,17,5.500000,150.0,False,False,True,False,False,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,False,16,5.333333,140.0,False,False,False,False,False,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,False,19,5.833333,150.0,False,False,False,False,False,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,False,35,6.000000,200.0,False,False,True,False,False,False,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,False,20,5.666667,160.0,False,True,False,False,True,True,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
col = X.dropna(axis=1).columns

In [46]:
for c in X.columns:
    if c not in col:
        print(c)

suspect.weight
lat
lon
